<p>
<img src="https://alanattableau.files.wordpress.com/2014/12/cropped-databender4.png" align=center>
<h1 align=center>The Last Databenders</h1>
<h4 align=center>Sponsored By Gutenberg</h4>
<img src="http://www.gutenberg.org/pics/logo-144x144.png" align=center>
</p>

In [1]:
# Remove the unnecessary text from files.
# Word groups which are identifying beginning and the end of necessary text.

# Word groups to find the beginning of text
HEADER_MARKERS = [

      '* START OF THE PROJECT GUTENBERG',
      '* START OF THIS PROJECT GUTENBERG',
      'This etext was prepared by',
      'E-text prepared by',
      'Produced by',
      'Distributed Proofreading Team',
      '*END THE SMALL PRINT',
      '***START OF THE PROJECT GUTENBERG',
      'This etext was produced by',
      '* START OF THE COPYRIGHTED',
      'The Project Gutenberg',
      'http://gutenberg.spiegel.de/ erreichbar.',
      'Project Runeberg publishes',
      'Beginning of this Project Gutenberg',
      'Project Gutenberg Online Distributed',
      'Gutenberg Online Distributed',
      'the Project Gutenberg Online Distributed',
      'Project Gutenberg TEI',
      'This eBook was prepared by',
      'http://gutenberg2000.de erreichbar.',
      'This Etext was prepared by',
      'Gutenberg Distributed Proofreaders',
      'the Project Gutenberg Online Distributed Proofreading Team',
      '**The Project Gutenberg',
      '*SMALL PRINT!',
      'More information about this book is at the top of this file.',
      'tells you about restrictions in how the file may be used.',
      'of the etext through OCR.',
      '**These eBooks Were Prepared By Thousands of Volunteers!**',
      'SERVICE THAT CHARGES FOR DOWNLOAD',
      'We need your donations more than ever!',
      ' * START OF THIS PROJECT GUTENBERG',
      '**     SMALL PRINT!'
    
]

# Word groups to find the end of the text.
FOOTER_MARKERS = [
    '* END OF THE PROJECT GUTENBERG',
    '* END OF THIS PROJECT GUTENBERG',
    '***END OF THE PROJECT GUTENBERG',
    'End of the Project Gutenberg',
    'End of The Project Gutenberg',
    'by Project Gutenberg',
    'End of Project Gutenberg',
    'End of this Project Gutenberg',
    '        ***END OF THE PROJECT GUTENBERG',
    '* END OF THE COPYRIGHTED',
    'End of this is COPYRIGHTED',
    '*This is a COPYRIGHTED Project Gutenberg Etext, Details Above*',
    'More information about this book is at the top of this file.',
    'We need your donations more than ever!',
    '<<THIS ELECTRONIC VERSION OF',
    'END OF PROJECT GUTENBERG',
    ' End of the Project Gutenberg',
    ' * END OF THIS PROJECT GUTENBERG',
    "THE END"  
]

In [2]:
# Necessary functions to get and transform the data into an easily processable dataset.

# the libs that handle the url stuff
import urllib2, string, json  
from bs4 import BeautifulSoup

# Standardization of the input data.
# Input type is string and represents data.
def ParseBook(data):
    is_header = False # Identifies if necessary text started.
    is_footer = False # Identifies if necessary text finished.
    for line in data:
        for footer in FOOTER_MARKERS:
            if footer in line.strip(): # Check if the current line includes a footer marker.
                is_footer = True # If footer marker found, sign the line as the ending point and interrupt the process.
                break
        if is_footer:
            break
        if is_header: # If headers removed, then proceed.
            for i in line.split():
                # STANDARDIZATION PROCESS, word by word
                word = i.strip().lower().translate(None, string.punctuation)
                tx.write(word+" ") # One large dataset txt file on disk, that will be processed in ENGINE.

        for header in HEADER_MARKERS: # Check if the current line includes a header marker.
            if header in line.strip(): # If footer marker found, sign the line as the beginning point and allow standardization process.
                is_header = True
                break

# Take the book from the URL.
# Inputs are an Integer and a String represents the ındex of the book and the category of the book respectively.                
def FillDict(index, cat):
    tx.write(cat+"\t") # Mark the book with its category; tab-separeted structure.
    try:
        data = urllib2.urlopen("http://www.gutenberg.org/files/"+str(index)+"/"+str(index)+"-0.txt") # it's a file like object and works just like a file
        ParseBook(data)
    except urllib2.HTTPError:
        try:
            data = urllib2.urlopen("http://www.gutenberg.org/files/"+str(index)+"/"+str(index)+".txt") # it's a file like object and works just like a file
            ParseBook(data)
        except:
            pass
    tx.write("\n") # Separate each book via new-line operator.

# Create a dataset of books; specified here as only children books
### Our script also calls this function for random categories to create a training dataset, 
### those codes are not included here.
def FindBookIndex():
    categorise = ['Children']
    for cat in categorise: 
        book_index = 0
        while book_index < 100: # The size of the dataset can be specified here.
            try: # Some categories could include less number of books than expected and raises HTTPError.
                html_doc = urllib2.urlopen("http://www.gutenberg.org/ebooks/search/?start_index="+str(book_index)+"&query="+cat+"+l.english").read()
                soup = BeautifulSoup(html_doc, 'html.parser')
                lis = soup.find_all("li",  { "class" : "booklink" })
                for i in range(len(lis)):
                    index = lis[i].a['href'].split("/")[2]
                    FillDict(index, cat)
                book_index += 26
            except urllib2.HTTPError: # Handling
                break
                
tx = open("main_Children.txt", "w") # Our main Training Data File. 
                                    # It is persisten on disk as txt so that we do not need to run this script each time.
FindBookIndex()
tx.close()
print "DONE"

Children
DONEE
